In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.pydantic_v1 import BaseModel, Field
import pickle
import os
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector

from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

from typing import Tuple, List, Optional

from langchain_ollama import ChatOllama
import torch

/home/mouli/miniconda3/envs/fusgraph/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
NEO4J_URI="neo4j+s://9269fc71.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="lOnpSSiAFQg0uWzA1EvYYBhK45_rWbxXb9SHRDwZRfk"

In [3]:
# Configure Neo4j and load embeddings
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [4]:
graph = Neo4jGraph()

In [5]:
graph_documents = Neo4jGraph(
    url=os.environ["NEO4J_URI"], username=os.environ["NEO4J_USERNAME"], password=os.environ["NEO4J_PASSWORD"]
)

In [10]:
embedding_device = torch.device("cuda:0")  # GPU 0 for embedding model
llm_device = torch.device("cuda:1")

In [14]:
embedding_device

device(type='cuda', index=0)

In [15]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": embedding_device.type}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

/tmp/ipykernel_5215/4173282442.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)
/home/mouli/miniconda3/envs/fusgraph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [17]:
llm = ChatOllama(model='llama3.2', temperature=0, num_gpu=1)

In [17]:
# llm = OllamaLLM(model="llama3.2")

In [18]:
fusion_template = """
        Task: You are an assistant that generates multiple variations of a given question. 
        For each variation, maintain the original intent of the question, but change the phrasing, structure, 
        or tone to create a diverse set of queries.

Generate 5-7 variations that cover:

Synonym replacements while keeping the question concise.
Alternative structures, such as rephrasing into "why," "how," or "what" forms if relevant.
Casual and formal tones.
Slightly more specific or broader wording.
Examples:

Original Question: "How is a gastric ulcer diagnosed?"
Variations:
    "What tests can be used to detect stomach ulcers?"
    "Why are endoscopies commonly used in gastric ulcer diagnosis?"
    "How do doctors figure out if someone has a stomach ulcer?"
    "Can an upper endoscopy be used to diagnose all types of gastric ulcers, or are there other methods more effective for certain cases?"
    "What tests and procedures are typically involved in diagnosing gastric ulcers, including acid reflux disease?"
    "How can a patient determine if they have been diagnosed with a stomach ulcer based on symptoms, test results, or imaging studies?"
    "When considering the diagnosis of a gastric ulcer, what role does symptom severity play in determining the need for further testing or treatment?"

Now generate just the list of variations one on each line for the given question and nothing else
        """

In [19]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template=fusion_template)),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (5 queries):'))])
     

In [20]:
original_query = 'How do NSAIDs induce gastric mucosal injury leading to ulcers?'

In [21]:
generate_queries = prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))

In [22]:
generate_queries.invoke('What causes gastric ulcers?')

['What triggers stomach ulcers?',
 'How do gastric ulcers form in the body?',
 'What are the common causes of stomach ulcers?',
 'Can stress contribute to the development of gastric ulcers?',
 'What medical conditions increase the risk of developing a gastric ulcer?']

In [23]:
# Define the schema without validators to avoid deepcopy issues
class Entities(BaseModel):
    """Identifying information about entities."""

    nodes: List[str] = Field(
        ...,
        description="All the Disease, Cause, Symptom, Complication, Treatment, Medication, Test, Risk Factor, Diagnosis, Side Effect, Procedure, Condition Stage, Disease Type, Imaging Type"
                    "that appear in the text"
    )

In [24]:
# Define the prompt template with instructions for node extraction
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are tasked with extracting the diseases, conditions, Diagnosis or symptoms from the qiven question "
            " for example, for the question: How is a gastric ulcer diagnosed?, you are going to find that gastric ulcer is a dieases and you also found that diagnosis is used in the question, so you are going to return gastric ulcer"
            # "You are tasked with extracting Disease, Cause, Symptom, Complication, Treatment, Medication, Test, Risk Factor, Diagnosis, Side Effect, Procedure, Condition Stage, Disease Type, Imaging Type from the text"
        ),
        (
            "human",
            "Use the specified format to extract information from the following input: {question}."
        ),
    ]
)

In [25]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [26]:
entity_chain.invoke({"question": "How do NSAIDs induce gastric mucosal injury leading to ulcers?"})

Entities(nodes=['Gastric ulcer', 'NSAID-induced gastric mucosal injury'])

In [27]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [50]:
# def structured_retriever(question: str) -> str:
#     result = ""
#     # Extract entities
#     entities = entity_chain.invoke({"question": question})
#     if not entities.nodes:
#         raise ValueError(f"No entities extracted from the question: {question}")
    
#     # Query each entity
#     for entity in entities.nodes:
#         query = generate_full_text_query(entity)
#         if not query:
#             raise ValueError(f"Generated an empty full-text query for entity: {entity}")
        
#         # Execute the full-text query
#         response = graph.query(
#             """
#             CALL db.index.fulltext.queryNodes('entityIndex', $query)
#             YIELD node, score
#             RETURN node.name AS name, score
#             """,
#             {"query": query}
#         )
        
#         # Handle no results
#         if not response:
#             result += f"No results found for entity: {entity}\n"
#         else:
#             result += "\n".join([f"Entity: {record['name']}, Score: {record['score']}" for record in response])
#     return result


In [28]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.nodes:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query)
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [29]:
print(structured_retriever("What causes gastric ulcers?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 93} for query: "CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output\n            "


Gastric ulcer - ASSOCIATED_WITH -> Bleeding
Gastric ulcer - ASSOCIATED_WITH -> Peptic ulcer disease
Gastric ulcer - ASSOCIATED_WITH -> Mass
Gastric ulcer - HAS_SYMPTOM -> Vomiting
Gastric ulcer - HAS_SYMPTOM -> Nausea
Gastric ulcer - HAS_SYMPTOM -> Pain soon after meals
Gastric ulcer - HAS_SYMPTOM -> Weight loss
Gastric ulcer - HAS_SYMPTOM -> Anorexia
H2RAs - TREATED_WITH -> Gastric ulcer
PPIs - TREATED_WITH -> Gastric ulcer
Sucralfate - TREATED_WITH -> Gastric ulcer
Misoprostol - TREATED_WITH -> Gastric ulcer
Proton pump inhibitors - TREATED_WITH -> Gastric ulcer
P-CAB - TREATED_WITH -> Gastric ulcer
Pirenzepine - TREATED_WITH -> Gastric ulcer
proton pump inhibitors - TREATED_WITH -> gastric ulcer
Gastric Ulcer - HAS_SYMPTOM -> ulcer stenosis; pyloric stenosis; oesophageal gastric varices; Barrett's oesophagus measuring >3 cm; intractable ulcer; digestive ulcer perforation or malignancy on upper GI endoscopy
Gastric Ulcer - HAS_SYMPTOM -> GI symptoms
Gastric Ulcer - COMPLICATES -> adv

In [30]:
def combined_retriever(queries):
    combined_results = []
    for query in queries:
        structured_data = structured_retriever(query)
        unstructured_data = [
            doc.page_content for doc in vector_index.similarity_search(query)
        ]
        combined_results.append({"structured": structured_data, "unstructured": "#Document ". join(unstructured_data)})
    return combined_results

In [31]:
# Combining results and generating the final response
def combined_fusion_graph_rag(original_query):
    # Step 1: Generate query variations
    # raw_output = generate_queries.invoke(original_query)

    # # Debugging: print raw output
    # print("Raw Output from generate_queries:", raw_output)

    # # Extract queries from raw output
    # queries = []
    # for line in raw_output:
    #     line = line.strip()
    #     if line.startswith('"') and line.endswith('",'):
    #         # Remove trailing comma and quotes
    #         queries.append(line.strip('",'))
    #     elif line.startswith('"') and line.endswith('"'):
    #         queries.append(line.strip('"'))

    # # Debugging: print parsed queries
    # print("Parsed Queries:", queries)

    queries = generate_queries.invoke(original_query)
    
    # Step 2: Retrieve data for each query
    retrieved_data = combined_retriever(queries)
    
    # Step 3: Combine retrieved results
    combined_context = "\n".join(
        f"Structured:\n{result['structured']}\nUnstructured:\n{' '.join(result['unstructured'])}"
        for result in retrieved_data
    )
    
    # Step 4: Generate final response
    answer_template = """Answer the question based only on the following context:
    {context}
    
    Question: {question}
    """
    final_prompt = ChatPromptTemplate.from_template(answer_template)
    chain = final_prompt | llm | StrOutputParser()
    return chain.invoke({"context": combined_context, "question": original_query})

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [54]:
# answers = []

In [137]:
sample = "What are the differences between gastritis, gastric erosion, and gastric ulcers?"

In [138]:
%%time
final_result = combined_fusion_graph_rag(sample)
print(final_result)
answers.append(final_result)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 93} for query: "CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.Fe

The text does not explicitly compare and contrast gastritis, gastric erosion, and gastric ulcers. However, based on general medical knowledge, here is a summary of their differences:

1. **Gastritis**: Inflammation of the stomach lining (mucosa) due to various causes such as infection, injury, or irritation. It can be acute or chronic and may not always produce symptoms.
2. **Gastric Erosion**: A superficial lesion on the gastric mucosa caused by acid-pepsin damage, often resulting from a peptic ulcer disease (PUD). Gastric erosion is typically asymptomatic but can lead to bleeding if it erodes into a blood vessel.
3. **Gastric Ulcer**: A deeper lesion in the stomach lining that results from chronic inflammation and damage caused by acid-pepsin. Gastric ulcers are more likely to cause symptoms such as abdominal pain, nausea, vomiting, and weight loss.

Key differences:

*   Gastritis is an inflammatory condition, while gastric erosion and ulceration involve tissue damage.
*   Gastric e

In [141]:
print(answers[24])

The text does not explicitly compare and contrast gastritis, gastric erosion, and gastric ulcers. However, based on general medical knowledge, here is a summary of their differences:

1. **Gastritis**: Inflammation of the stomach lining (mucosa) due to various causes such as infection, injury, or irritation. It can be acute or chronic and may not always produce symptoms.
2. **Gastric Erosion**: A superficial lesion on the gastric mucosa caused by acid-pepsin damage, often resulting from a peptic ulcer disease (PUD). Gastric erosion is typically asymptomatic but can lead to bleeding if it erodes into a blood vessel.
3. **Gastric Ulcer**: A deeper lesion in the stomach lining that results from chronic inflammation and damage caused by acid-pepsin. Gastric ulcers are more likely to cause symptoms such as abdominal pain, nausea, vomiting, and weight loss.

Key differences:

*   Gastritis is an inflammatory condition, while gastric erosion and ulceration involve tissue damage.
*   Gastric e

In [140]:
len(answers)

25

In [142]:
import pickle
with open('sample', 'wb') as f:
    pickle.dump(answers, f)

In [143]:
questions = [
    "What is a gastric ulcer?",
    "What causes gastric ulcers?",
    "How does H. pylori infection contribute to gastric ulcers?",
    "What are the common symptoms of a gastric ulcer?",
    "How is a gastric ulcer diagnosed?",
    "What tests are used to detect H. pylori infection?",
    "What are the common treatments for gastric ulcers?",
    "What are the potential side effects of ulcer medications?",
    "How do proton pump inhibitors (PPIs) work to treat gastric ulcers?",
    "What are the long-term effects of untreated gastric ulcers?",
    "What are the signs that a gastric ulcer is bleeding?",
    "What should I do if I suspect my gastric ulcer is bleeding?",
    "How is a bleeding gastric ulcer treated in an emergency?",
    "What are the symptoms of a perforated gastric ulcer?",
    "How is a perforated gastric ulcer treated?",
    "How does anemia relate to gastric ulcers?",
    "What are the alternatives to NSAIDs if I have a history of gastric ulcers?",
    "How does gastric acid secretion influence the formation of stomach ulcers?",
    "How is the urea breath test used to diagnose H. pylori infection?",
    "How do NSAIDs induce gastric mucosal injury leading to ulcers?",
    "What is the role of endoscopy in the management of peptic ulcer disease?",
    "How does chronic use of corticosteroids influence peptic ulcer formation?",
    "How do you differentiate between benign and malignant gastric ulcers during endoscopy?",
    "What are the indications for endoscopic biopsy in patients with suspected gastric ulcers?",
    "What are the differences between gastritis, gastric erosion, and gastric ulcers?"
]

In [145]:
import pandas as pd

In [147]:


# Save to Excel
data = {"Question": questions, "Answer": answers}
df = pd.DataFrame(data)
output_path = "gastric_ulcer_answers.xlsx"
df.to_excel(output_path, index=False)
df.to_csv('fus_graph_output.csv', index=False)

print(f"Answers saved to {output_path}")

Answers saved to gastric_ulcer_answers.xlsx


In [148]:
df

,Question,Answer
0,What is a gastric ulcer?,A gastric ulcer is a type of peptic ulcer that...
1,What causes gastric ulcers?,Gastric ulcers can be caused by several factor...
2,How does H. pylori infection contribute to gas...,H. pylori infection is believed to contribute ...
3,What are the common symptoms of a gastric ulcer?,The question is not directly related to the pr...
4,How is a gastric ulcer diagnosed?,A gastric ulcer can be diagnosed through vario...
5,What tests are used to detect H. pylori infect...,"According to the text, the following tests are..."
6,What are the common treatments for gastric ulc...,"Based on the provided text, the following are ..."
7,What are the potential side effects of ulcer m...,"The text does not explicitly state ""ulcer medi..."
8,How do proton pump inhibitors (PPIs) work to t...,Proton pump inhibitors (PPIs) are medications ...
9,What are the long-term effects of untreated ga...,The text does not explicitly state the long-te...


In [71]:
output_path = "gastric_ulcer_answers.xlsx"
df.to_excel(output_path, index=False)